In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
from html.parser import HTMLParser
from html.entities import name2codepoint
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")                   
import nltk                                         
nltk.download('stopwords')
from nltk.corpus import stopwords                   
from nltk.stem import PorterStemmer  
LA = np.linalg

from sklearn.feature_extraction.text import CountVectorizer          
from sklearn.feature_extraction.text import TfidfVectorizer    
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from collections import defaultdict
from gensim.models.word2vec import Word2Vec                                  
import re
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chd415/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load training data text
def load_data(filename):
    load_file = pd.read_csv(filename,delimiter=',', header=0,
                        dtype={'name':str, 'lvl1':str, 'lvl2':str, 'lvl3':str, 'descrption':str, 'type':str})
    load_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
    load_file.duplicated(subset=None, keep='first')
    load_file.set_index('id', inplace = True)
    load_file.head()
    return load_file
#print(len(train_file))
def load_label(filename):
    load_label = pd.read_csv(filename,delimiter=',', header=0)
    load_label.columns = ['id', 'score']
    load_label.duplicated(subset=None, keep='first')
    load_label.set_index('id', inplace = True)
    return load_label

In [3]:
'''
def map_mathod(column):
    values = []
    indexs = []
    mapping = {}
    index = 0
    for count in range(len(train_file)):
        value = train_file.get_value(count+1,column)
        if value in values and value != np.nan:
            continue
        values.append(value)
        indexs.append(len(values))
    for j in range(len(indexs)):
        mapping[values[j]] = indexs[j]
    mapping[np.nan] = 0.0
    return mapping
#train_file['lvl3'] = train_file['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
#mapping_lvl3 = map_mathod('lvl3')
#print(mapping_lvl3)
'''

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)


In [4]:
def text_embedding(column):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=10000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(max_features=10000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=1,size=50, workers=4)
    avg_data = []
    for row in splitted:
        vec = np.zeros(50, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)
#        avg_data.append(vec)
    
    return avg_data

In [ ]:
'''
#test cell
temp = load_data('train_data.csv')
temp['descrption'] = temp['descrption'].str.lower()
description_X = temp.descrption.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
'''

In [ ]:
'''
#test cell
def test_embedding(column):
    temp_X = column.astype(str)
    stop = set(stopwords.words('english'))
    temp =[]
    snow = nltk.stem.SnowballStemmer('english')

    for sentence in temp_X: 
        words = [snow.stem(word) for word in sentence.split(' ') if word not in stopwords.words('english')]   # Stemming and removing stopwords
        temp.append(sentence)

    count_vect = CountVectorizer(max_features=5000)
    bow_data = count_vect.fit_transform(temp)

    final_tf = temp
    tf_idf = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english',max_features=5000)
    tf_data = tf_idf.fit_transform(final_tf)
    w2v_data = temp
    splitted = []
    for row in w2v_data: 
        splitted.append([word for word in row.split()])     #splitting words
    
    train_w2v = Word2Vec(splitted,min_count=5,size=50, workers=4)
    avg_data = []
    for row in splitted:
        vec = np.zeros(50, dtype=float)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        if (count == 0):
            avg_data.append(vec)
        else:
            avg_data.append(vec/count)
#        avg_data.append(vec)

    tf_w_data = []
    tf_data = tf_data.toarray()
    i = 0
    for row in splitted:
        vec = [0 for i in range(50)]
    
        temp_tfidf = []
        for val in tf_data[i]:
            if val != 0:
                temp_tfidf.append(val)
    
        count = 0
        tf_idf_sum = 0
        for word in row:
            try:
                count += 1
                tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
                vec += (temp_tfidf[count-1] * train_w2v[word])
            except:
                pass
        if (tf_idf_sum == 0):
            tf_w_data.append(vec)
        else:
            tf_w_data.append(vec/tf_idf_sum)
#            vec = float(1/tf_idf_sum) * vec
#        tf_w_data.append(vec)
        i = i + 1
    
    return tf_w_data
'''

In [ ]:
'''
#test cell
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

temp['descrption'] = test_embedding(description_X)
temp.descrption.head()
#tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')
'''

In [5]:
def clean_data(filename):
    
    filename['lvl1'] = filename['lvl1'].str.lower().replace('[^a-zA-Z]+',' ',regex=True)
    filename['lvl2'] = filename['lvl2'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['lvl3'] = filename['lvl3'].str.lower().replace('[^\'\w]+',' ',regex=True)
    filename['descrption'] = filename['descrption'].str.lower()
    filename['name'] = filename['name'].str.lower()

    '''
    
    mapping_lvl1 = map_mathod('lvl1')
    mapping_lvl2 = map_mathod('lvl2')
    mapping_lvl3 = map_mathod('lvl3')
    
    filename['lvl1'] = filename['lvl1'].map(mapping_lvl1)
    filename['lvl2'] = filename['lvl2'].map(mapping_lvl2)
    filename['lvl3'] = filename['lvl3'].map(mapping_lvl3)
    
    '''
    #clean up data for lvl 1&2&3
    temp =  filename.drop(['price', 'descrption','name','type'], axis=1)
    outfile = MultiColumnLabelEncoder(columns = ['lvl1','lvl2','lvl3']).fit_transform(temp.astype(str))
    enc = preprocessing.OneHotEncoder()
    enc.fit(outfile)
    outfile = enc.transform(outfile).toarray()
    
    #normalize price
    maxp = filename.price.max()
    valuethred = 1000.
    filename['price'] = filename['price'].clip(lower=0.,upper=valuethred)
    #filename['price'] = filename['price'].clip(lower=0.,upper=valuethred).div(valuethred,fill_value=None)
    #hist = train_file['price'].hist(bins=10)
    #maxp
    ld = filename.price.as_matrix(columns=None).tolist()
    outfile = np.column_stack((outfile,ld))

    #clean up type 
    mapping_type = {'international':1.,'local':2., np.nan:0.}
    filename['type'] = filename['type'].map(mapping_type)
    le = filename.type.as_matrix(columns=None).tolist()
    outfile = np.column_stack((outfile,le))

    #clean up text
    description_X = filename.descrption.str.lower().replace('<li>','final ',regex=True).replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    count_descrption = description_X.str.count('final').fillna(0).tolist()
    outfile = np.column_stack((outfile,count_descrption))
    
    description_X = filename.descrption.str.lower().replace('final ','',regex=True)
    filename['descrption'] = text_embedding(description_X)
 
    lg = filename.descrption.as_matrix(columns=None).tolist()
    lg_array = np.vstack( lg )

    U, s, Vh = LA.svd(lg_array, full_matrices=False)
    assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))
    s_new = s[:8]
    U_new = U[:, :8]
    new_lg = np.dot(U_new, np.diag(s_new))
    lg_min = np.min(new_lg)
    lg_out = new_lg  - (lg_min-2)*np.ones_like(new_lg.size)
    outfile = np.column_stack((outfile,lg_out))
    
    
    name_X = filename.name.str.lower().replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
    filename['name'] = text_embedding(name_X)
    
    lf = filename.name.as_matrix(columns=None).tolist()
    lf_array = np.vstack( lf )

    Uf, sf, Vhf = LA.svd(lf_array, full_matrices=False)
    assert np.allclose(lf_array, np.dot(Uf, np.dot(np.diag(sf), Vhf)))
    sf_new = sf[:8]
    Uf_new = Uf[:, :8]
    new_lf = np.dot(Uf_new, np.diag(sf_new))
    lf_min = np.min(new_lf)
    lf_out = new_lf  - (lf_min-2)*np.ones_like(new_lf.size)
    outfile = np.column_stack((outfile,lf_out))
   

    return outfile

In [ ]:
'''
train_file = pd.read_csv('train_data.csv',delimiter=',', header=0, nrows=10)
train_file.columns = ['id', 'name','lvl1','lvl2','lvl3','descrption','price','type']
description_X = train_file.descrption.str.lower().replace('<li>','final ',regex=True).replace('<.*?>','',regex=True).replace('[^\w\s]+',' ',regex=True)
count_descrption = description_X.str.count('final')
description_X = train_file.descrption.str.lower().replace('final ','',regex=True)
print(np.shape(count_descrption))

'''


In [6]:
train_file = load_data('train_data.csv')
test_file = load_data('test_data.csv')
combined_file = pd.concat([train_file,test_file])
cleaned_train = clean_data(combined_file)
train_score = load_label('train_label.csv')
np.shape(cleaned_train)

(36283, 270)

In [ ]:
'''
# test cell
cleaned_train['descrption'] = temp['descrption']
'''

In [7]:
cleaned_train
#print(mapping_lvl1)


array([[ 0.        ,  0.        ,  0.        , ..., 10.13324131,
        11.99554897, 11.35688418],
       [ 0.        ,  1.        ,  0.        , ..., 11.9036486 ,
         9.82575646, 10.58128699],
       [ 0.        ,  0.        ,  0.        , ..., 11.73378475,
        11.62101979, 11.51830028],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., 12.33374908,
        11.35996205, 11.5136934 ],
       [ 0.        ,  0.        ,  0.        , ..., 11.17770531,
        11.97945511, 11.61931454],
       [ 0.        ,  0.        ,  0.        , ..., 10.76861269,
        11.21852384, 11.04587675]])

In [8]:
'''

def rearrange(cleaned_data):
    la = cleaned_data.lvl1.as_matrix(columns=None).tolist()
    lb = cleaned_data.lvl2.as_matrix(columns=None).tolist()
    lc = cleaned_data.lvl3.as_matrix(columns=None).tolist()

    X = la
    X = np.column_stack((X,lb))
    X = np.column_stack((X,lc))

    enc = preprocessing.OneHotEncoder()
    enc.fit(X)
    X = enc.transform(X).toarray()
    
    ld = cleaned_data.price.as_matrix(columns=None).tolist()
    le = cleaned_data.type.as_matrix(columns=None).tolist()

    X = np.column_stack((X,ld))
    X = np.column_stack((X,le))

   
    lf = cleaned_data.name.as_matrix(columns=None).tolist()
    lg = cleaned_data.descrption.as_matrix(columns=None).tolist()
    lg_array = np.vstack( lg )
    lf_array = np.vstack( lf )

    U, s, Vh = LA.svd(lg_array, full_matrices=False)
    assert np.allclose(lg_array, np.dot(U, np.dot(np.diag(s), Vh)))

# only use U \cdot s
    
#    s[8:] = 0.
#    new_lg = np.dot(U, np.dot(np.diag(s), Vh))
    s_new = s[:5]
    U_new = U[:, :5]
    new_lg = np.dot(U_new, np.diag(s_new))
    lg_min = np.min(new_lg)
    lg_out = new_lg  - (lg_min-2)*np.ones_like(new_lg.size)

    Uf, sf, Vhf = LA.svd(lf_array, full_matrices=False)
    assert np.allclose(lf_array, np.dot(Uf, np.dot(np.diag(sf), Vhf)))

#    sf[5:] = 0.
#    new_lf = np.dot(Uf, np.dot(np.diag(sf), Vhf))
    sf_new = sf[:5]
    Uf_new = Uf[:, :5]
    new_lf = np.dot(Uf_new, np.diag(sf_new))
    lf_min = np.min(new_lf)
    lf_out = new_lf  - (lf_min-2)*np.ones_like(new_lf.size)

    
    X = np.column_stack((X,lf_out))
    X = np.column_stack((X,lg_out))
    X = X.tolist()
   
    return X,lf_min,lg_min
'''
    
    #print(len(X))
X = cleaned_train#rearrange(cleaned_train)
print(np.shape(np.array(X)))
Y = train_score.score.as_matrix(columns=None).tolist()
X[137,253]

(36283, 270)


0.0

In [9]:
X = cleaned_train[:18141]
XX = cleaned_train[18141:]
Y = train_score.score.as_matrix(columns=None).tolist()
print(np.size(Y))
print(np.shape(X))
print(np.shape(XX))


18141
(18141, 270)
(18142, 270)


In [10]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y)
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, test_size=0.20, random_state=0)

from keras.preprocessing import sequence
maxlen = 270
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, dtype='float32')
X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, dtype='float32')
#X_train = np.any(np.isnan(X_train))
#X_train = np.all(np.isfinite(X_train))
print(X_train[1400].size)

Using TensorFlow backend.


270


In [11]:
import os
import time
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import math
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import cross_val_score

In [12]:
# Linear Regression Classifier
def linear_regression_classifier(train_x, train_y):
    model = linear_model.LinearRegression()
    model.fit(train_x, train_y)
    return model
 
# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    model = MultinomialNB()

    param_grid = {'alpha': [math.pow(10,-i) for i in range(11)]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = MultinomialNB(alpha = best_parameters['alpha'])  
    model.fit(train_x, train_y)
    return model
 
 
# KNN Classifier
def knn_classifier(train_x, train_y):
    model = KNeighborsClassifier()

    param_grid = {'n_neighbors': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = KNeighborsClassifier(n_neighbors = best_parameters['n_neighbors'], algorithm='kd_tree')

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model
 
 
# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(1,21))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])
    
    model.fit(train_x, train_y)
    return model
 
 
# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)

    bagging = BaggingClassifier(model, max_samples=0.5, max_features=1 )
    bagging.fit(train_x, train_y)
    return bagging
 
 
# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    model = GradientBoostingClassifier()
    
    model = RandomForestClassifier()

    param_grid = {'n_estimators': list(range(100,300,10))}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    
    model = RandomForestClassifier(n_estimators = best_parameters['n_estimators'])

    model.fit(train_x, train_y)
    return model

# SVM Classifier
def svm_classifier(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    model.fit(train_x, train_y)
    return model
 
# SVM Classifier using cross validation
def svm_cross_validation(train_x, train_y):
    model = SVC(kernel='linear', probability=True)
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    #for para, val in best_parameters.items():
        #print para, val
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)
    model.fit(train_x, train_y)
    return model

def feature_select(x,y):
    clf = ExtraTreesClassifier()
    clf = clf.fit(x, y)
    model = SelectFromModel(clf, prefit=True)
    return model

In [13]:
# just for my own record


if __name__ == '__main__': 
    thresh = 0.5    
#    model_save_file = "/home/jason/datamining/model/models"     
#    model_save = {}
#    result_save_file = '/home/jason/datamining/result/results' 
     
    test_classifiers = ['KNN','LR','RF','DT','GBC']    
    classifiers = {
                   'KNN':knn_classifier,
                    'LR':logistic_regression_classifier,
                    'RF':random_forest_classifier,
                    'DT':decision_tree_classifier,
                    'GBC':gradient_boosting_classifier
    }
        
    print('reading training and testing data...')    
    #X_train, X_validation, y_train, y_validation
    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_validation = select_model.transform(X_validation)

    result = []
        
    for classifier in test_classifiers:    
        print('******************* %s ********************' % classifier)    
        start_time = time.time()    
        model = classifiers[classifier](X_train, y_train)   
        print('training took %fs!' % (time.time() - start_time))    
        predict = model.predict(X_validation)

        precision = metrics.precision_score(y_validation, predict)    
        recall = metrics.recall_score(y_validation, predict)    
        print('precision: %.2f%%, recall: %.2f%%' % (100 * precision, 100 * recall))    
        accuracy = metrics.accuracy_score(y_validation, predict)    
        print('accuracy: %.2f%%' % (100 * accuracy))

        scores = cross_val_score(model, X_train, y_train)
        print(scores)

reading training and testing data...
******************* KNN ********************
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   16.6s finished


training took 16.900737s!
precision: 69.16%, recall: 100.00%
accuracy: 69.17%
[0.6920215  0.69588588 0.69051065]
******************* LR ********************
training took 0.241711s!
precision: 77.36%, recall: 90.43%
accuracy: 75.09%
[0.72219926 0.7366136  0.73433947]
******************* RF ********************
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   19.7s finished


training took 21.534177s!
precision: 79.11%, recall: 89.36%
accuracy: 76.33%
[0.73770153 0.7506719  0.74260906]
******************* DT ********************
training took 0.650436s!
precision: 71.19%, recall: 92.07%
accuracy: 68.75%
[0.68561389 0.66694232 0.68658259]
******************* GBC ********************
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  6.4min finished


training took 410.052062s!
precision: 79.57%, recall: 90.35%
accuracy: 77.29%
[0.75010335 0.76431673 0.76121563]


In [14]:
X_train
#X_train, y_train = X,Y

array([[ 0.      ,  1.      ,  0.      , ..., 11.774629, 11.737314,
        11.739768],
       [ 0.      ,  0.      ,  0.      , ..., 11.737541, 12.21775 ,
        11.256157],
       [ 1.      ,  0.      ,  0.      , ..., 11.562332, 11.451252,
        11.531164],
       ...,
       [ 0.      ,  0.      ,  1.      , ..., 11.248144, 11.919338,
        12.642514],
       [ 0.      ,  1.      ,  0.      , ..., 11.222754, 11.798913,
        11.439027],
       [ 0.      ,  1.      ,  0.      , ..., 11.568027, 11.701103,
        11.861883]], dtype=float32)

In [15]:
#test cnn model

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

# Embedding
max_features = 15000
maxlen = 270
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 3

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 5

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')
#X_train, X_validation, y_train, y_validation
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

#X_train = np.asarray(np.abs(X))
X_train = np.asarray(np.abs(X_train))
X_validation = np.asarray(np.abs(X_validation))
print(len(X_train), 'train sequences')
print(len(X_validation), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_validation = sequence.pad_sequences(X_validation, maxlen=maxlen, padding='post')
print('x_train shape:', X_train.shape)
print('x_test shape:', X_validation.shape)

print('Build model...')

cnnmodel = Sequential()
cnnmodel.add(Embedding(max_features, embedding_size,input_length=maxlen))
#model.add(Dense(32, activation='relu', input_dim=100))
cnnmodel.add(Dropout(0.5))
cnnmodel.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
cnnmodel.add(MaxPooling1D(pool_size=pool_size))
cnnmodel.add(LSTM(lstm_output_size))
cnnmodel.add(Dense(1))
cnnmodel.add(Activation('sigmoid'))

cnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Loading data...
14512 train sequences
3629 test sequences
Pad sequences (samples x time)
x_train shape: (14512, 270)
x_test shape: (3629, 270)
Build model...


In [16]:
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()

In [17]:
if __name__ == '__main__': 
    thresh = 0.5    
#    model_save_file = "/home/jason/datamining/model/models"     
#    model_save = {}
#    result_save_file = '/home/jason/datamining/result/results' 
     
    test_classifiers = ['LR','RF','GBC']    
    classifiers = {
                   'LR':logistic_regression_classifier,
                   'RF':random_forest_classifier,
                   'GBC':gradient_boosting_classifier
    }
        
    print('reading training and testing data...')    
    #X_train, X_validation, y_train, y_validation
#    X_test = rearrange(Xt)
#    X_test = Xt
    select_model = feature_select(X_train, y_train)
    X_train = select_model.transform(X_train)
    X_test = select_model.transform(X_test)

    result = []
    
    start_time = time.time()
    model = classifiers['LR'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_lr = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
    model = classifiers['RF'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_rf = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
    model = classifiers['GBC'](X_train, y_train)   
    print('training took %fs!' % (time.time() - start_time))    
    Y_predict_gbc = model.predict_proba(X_test)[:,1]
    print('predict finished')
    
        
#    for classifier in test_classifiers:    
#        print('******************* %s ********************' % classifier)    
#        start_time = time.time()    
#        model = classifiers[classifier](X_train, y_train)   
#        print('training took %fs!' % (time.time() - start_time))    
#        Y_predict = model.predict_proba(X_test)[:,1]
#        print('predict finished')

reading training and testing data...
training took 0.241209s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   25.5s finished


training took 27.739618s!
predict finished
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  8.2min finished


training took 546.078951s!
predict finished


In [18]:
#cnn test
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()
X_train = np.asarray(X_train)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
#X_test = np.asarray(Xt)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')


cnnmodel.fit(X_train, y_train, batch_size=128, epochs=5)

Y_predict_cnn = cnnmodel.predict(X_test, verbose=0)
Y_predict_cnn = np.squeeze(Y_predict_cnn, axis=1)

Epoch 1/5
18141/18141 [==============================] - 60s 3ms/step - loss: 0.5846 - acc: 0.6992
Epoch 2/5
18141/18141 [==============================] - 59s 3ms/step - loss: 0.5381 - acc: 0.7351
Epoch 3/5
18141/18141 [==============================] - 59s 3ms/step - loss: 0.5243 - acc: 0.7417
Epoch 4/5
18141/18141 [==============================] - 62s 3ms/step - loss: 0.5139 - acc: 0.7487
Epoch 5/5
18141/18141 [==============================] - 65s 4ms/step - loss: 0.5042 - acc: 0.7539


In [19]:
#nn train
X_train = cleaned_train[:18141]
X_test = cleaned_train[18141:]
y_train = train_score.score.as_matrix(columns=None).tolist()
X_array = np.asarray(X_train)
Y_array = np.asarray(y_train)
Xtest_array = np.asarray(X_test) 
import warnings
warnings.filterwarnings("ignore")   
# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense


# fix random seed for reproducibility
np.random.seed(7)

# split into input (X) and output (Y) variables
X = X_array
Y = Y_array
# create model
nnmodel = Sequential()
nnmodel.add(Dense(100, input_dim=270, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(100, activation='relu'))
nnmodel.add(Dense(1, activation='sigmoid'))
# Compile model
nnmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
nnmodel.fit(X, Y, epochs=25, batch_size=150)
# evaluate the model
scores = nnmodel.evaluate(X, Y)
print("\n%s: %.2f%%" % (nnmodel.metrics_names[1], scores[1]*100))
Y_predict_nn = nnmodel.predict(Xtest_array, verbose=0)
Y_predict_nn = np.squeeze(Y_predict_nn, axis=1)

Epoch 1/25
18141/18141 [==============================] - 1s 59us/step - loss: 0.6737 - acc: 0.6873
Epoch 2/25
18141/18141 [==============================] - 0s 26us/step - loss: 0.5794 - acc: 0.7171
Epoch 3/25
18141/18141 [==============================] - 0s 22us/step - loss: 0.5597 - acc: 0.7253
Epoch 4/25
18141/18141 [==============================] - 0s 22us/step - loss: 0.5516 - acc: 0.7259
Epoch 5/25
18141/18141 [==============================] - 0s 23us/step - loss: 0.5565 - acc: 0.7264
Epoch 6/25
18141/18141 [==============================] - 0s 22us/step - loss: 0.5491 - acc: 0.7297
Epoch 7/25
18141/18141 [==============================] - 1s 29us/step - loss: 0.5310 - acc: 0.7361
Epoch 8/25
18141/18141 [==============================] - 0s 23us/step - loss: 0.5338 - acc: 0.7349
Epoch 9/25
18141/18141 [==============================] - 0s 26us/step - loss: 0.5247 - acc: 0.7377
Epoch 10/25
18141/18141 [==============================] - 0s 23us/step - loss: 0.5242 - acc: 0.7415

In [20]:
Y_predict_nn

array([0.8579925 , 0.94041866, 0.8313853 , ..., 0.70595163, 0.50523454,
       0.61891323], dtype=float32)

In [21]:
Y_predict_cnn

array([0.8295582 , 0.78621113, 0.6645141 , ..., 0.64757854, 0.41389206,
       0.569912  ], dtype=float32)

In [22]:
Y_predict_lr

array([0.73480094, 0.67819865, 0.68934352, ..., 0.62148617, 0.41585488,
       0.49107215])

In [23]:
Y_predict_rf

array([0.77777778, 0.83333333, 0.61111111, ..., 0.66666667, 0.27777778,
       0.72222222])

In [24]:
Y_predict_gbc

array([0.68636364, 0.79545455, 0.71818182, ..., 0.68636364, 0.52272727,
       0.58636364])

In [25]:
Y_predict = []
for i in range(len(Y_predict_rf)):
    Y_predict.append(( Y_predict_rf[i] + Y_predict_gbc[i] + Y_predict_cnn[i] + Y_predict_nn[i]) / 4.0)

In [31]:
Y_predict

[0.7879230195223683,
 0.8388544181079576,
 0.7062980920377404,
 0.943190951750736,
 0.768477741034344,
 0.7789613797508105,
 0.49860520630773875,
 0.7843875995790115,
 0.7212234910690423,
 0.7845466396122267,
 0.8631969074709247,
 0.3807709932026237,
 0.6003489641830174,
 0.40475087120677483,
 0.8834678248323575,
 0.2311255061265194,
 0.873183729010399,
 0.6564867117489227,
 0.8245094442307347,
 0.5769025559527705,
 0.9085451350970701,
 0.8365589516632485,
 0.8573541283005416,
 0.8478584218506862,
 0.9570992681113156,
 0.6973043681997242,
 0.7669518869633627,
 0.9678706323558635,
 0.8102602714239949,
 0.9404498155670937,
 0.8150021161394889,
 0.7075678125926943,
 0.8056072395558309,
 0.6466429714602653,
 0.720919211645319,
 0.45648324354128406,
 0.6798729751748268,
 0.9523854328827425,
 0.5096632492963714,
 0.8012050654852029,
 0.5585386865367793,
 0.8966844571961297,
 0.9577842205762863,
 0.6449160896166406,
 0.5598847478327125,
 0.8654995611821762,
 0.7667021310690678,
 0.31053749369

In [27]:
Y_predict_temp = []
for i in range(len(Y_predict_rf)):
    Y_predict_temp.append((Y_predict_lr[i] + Y_predict_cnn[i]) / 2.0)

In [32]:
Y_predict_temp

[0.7821795656902211,
 0.7322048928074766,
 0.6769288237714355,
 0.9232060886053208,
 0.7636085474986847,
 0.6989003050179315,
 0.6514817259038301,
 0.7032082072973302,
 0.7123131421092022,
 0.686663417226966,
 0.9097344714158052,
 0.4704497726606686,
 0.6847283869564542,
 0.5155052038948547,
 0.915586192621144,
 0.26201000179906697,
 0.8994400718614441,
 0.6368460209090437,
 0.7858382623603368,
 0.3839700593026235,
 0.8084173498524736,
 0.8017286241164303,
 0.9136781189201648,
 0.8471509975068205,
 0.9566878157273446,
 0.6656232986516144,
 0.7254675937410695,
 0.942262732292676,
 0.8090296200115747,
 0.94187385551492,
 0.6098780237977018,
 0.7268148201302096,
 0.8395090462491981,
 0.6232918159426388,
 0.6252407055509681,
 0.43514013130343,
 0.7910901168834001,
 0.880788700237831,
 0.5815274392765865,
 0.833717457521943,
 0.6208477364940778,
 0.8997755945424697,
 0.8608512841555309,
 0.5341327947115535,
 0.5832420539201155,
 0.8050140912758983,
 0.7257269586465179,
 0.39700496960241305,

In [34]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict
submit_score.to_csv('predict_result_cnn.csv')

In [30]:
temp_score = load_label('submission.csv')
submit_score  = temp_score
submit_score['score'] = Y_predict_temp
submit_score.to_csv('predict_result_ncnn.csv')

In [ ]:
table = pd.pivot_table(train_file,values = 'price', index=['lvl1','lvl2','lvl3'],columns=['type'],aggfunc=[min, max, np.mean])
table